# Challenge 1 - Most Profiting Authors

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./ironhack_service_account_big_query.json"
import six
six.moves.reload_module(six)
from google.cloud import bigquery
client = bigquery.Client()

In [109]:
sql="""
SELECT
    A.au_id,
    A.au_lname, 
    A.au_fname,
    IFNULL(SUM(ROYALTYTA*TA.royaltyper/100),0) AS ROYALTY,
    IFNULL(SUM(T.advance*TA.royaltyper/100),0) AS ADVANCE,  
    IFNULL(SUM(ROYALTYTA*TA.royaltyper/100),0) + IFNULL(SUM(T.advance*TA.royaltyper/100),0) AS PROFIT
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` TA 
ON 
    A.au_id=TA.au_id
LEFT JOIN 
    `ironhack-data-analytics-265219.publications.titles` T 
ON
    TA.title_id=T.title_id
LEFT JOIN    
(
    SELECT
        TA.au_id, 
        TA.title_id, 
        IFNULL(SUM(T.price*S.qty*T.royalty/100),0) AS ROYALTYTA
    FROM
        `ironhack-data-analytics-265219.publications.titleauthor` TA 
    LEFT JOIN 
        `ironhack-data-analytics-265219.publications.titles` T 
    ON
        TA.title_id=T.title_id
    LEFT JOIN
        `ironhack-data-analytics-265219.publications.sales` S
    ON 
        T.title_id=S.title_id
    GROUP BY
        TA.au_id, 
        TA.title_id
) R
ON TA.au_id=R.au_id
AND TA.title_id=R.title_id
GROUP BY 
    A.au_id,
    A.au_lname, 
    A.au_fname
ORDER BY 
    PROFIT DESC
LIMIT 3
"""

In [110]:
query_job = client.query(query=sql)
df=query_job.to_dataframe()
df.head(70)

,au_id,au_lname,au_fname,ROYALTY,ADVANCE,PROFIT
0,213-46-8915,Green,Marjorie,37.110,12125.0,12162.110
1,722-51-5454,DeFrance,Michel,21.528,11250.0,11271.528
2,998-72-3567,Ringer,Albert,88.456,7137.5,7225.956


# Challenge 2

In [115]:
sql="""
CREATE TABLE publications.most_profiting_authors AS
SELECT
    A.au_id,
    IFNULL(SUM(ROYALTYTA*TA.royaltyper/100),0) + IFNULL(SUM(T.advance*TA.royaltyper/100),0) AS PROFIT
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` TA 
ON 
    A.au_id=TA.au_id
LEFT JOIN 
    `ironhack-data-analytics-265219.publications.titles` T 
ON
    TA.title_id=T.title_id
LEFT JOIN    
(
    SELECT
        TA.au_id, 
        TA.title_id, 
        IFNULL(SUM(T.price*S.qty*T.royalty/100),0) AS ROYALTYTA
    FROM
        `ironhack-data-analytics-265219.publications.titleauthor` TA 
    LEFT JOIN 
        `ironhack-data-analytics-265219.publications.titles` T 
    ON
        TA.title_id=T.title_id
    LEFT JOIN
        `ironhack-data-analytics-265219.publications.sales` S
    ON 
        T.title_id=S.title_id
    GROUP BY
        TA.au_id, 
        TA.title_id
) R
ON TA.au_id=R.au_id
AND TA.title_id=R.title_id
GROUP BY 
    A.au_id
ORDER BY 
    PROFIT DESC
LIMIT 3
"""

In [116]:
query_job = client.query(query=sql)
df=query_job.to_dataframe()
df.head(70)

""
